# Installation

With conda, you can install the required dependencies with:

```bash
conda install beautifulsoup4 requests
```


# Basic usage of BeautifulSoup

First, we import the `BeatifulSoup` class:

In [ ]:
from bs4 import BeautifulSoup

We load the html source file from disk and pass the source the the BeautifulSoup constructor.

In [ ]:
with open("list.html") as f:
    html = f.read()
    document = BeautifulSoup(html, "html.parser")
print(html)

In [ ]:
from IPython.display import HTML
HTML(html)

### Finding tags by name

The document now contains the full html document. We can find the first occuring tag with a specific name with the `find` function. Let's find the first un-ordered list tag:

In [ ]:
ulist = document.find("ul")

The result contains all tags contained in the matched tag:

In [ ]:
ulist

The `find_all` function returns **all** tags that match the given tag name. We can use it to get a list of all list items:

In [ ]:
items = ulist.find_all("li")
items

Finally, we can loop over all items and extract their contant with the `get_text` function:

In [ ]:
for item in items:
    print(repr(item.get_text()))

Because whitespace is not meaningful in HTML,
it is often useful to strip it when you are getting the content of a tag.
You can do this with `strip=True`

In [ ]:
for item in items:
    print(repr(item.get_text(strip=True)))

Note that `find_all` is **recursive** by default. This means that we could call it the on the full `document` to get the items
of both the ordered and un-ordered lists:

In [ ]:
document.find_all("li", recursive=False)

In [ ]:
ulist.find_all("li", recursive=False)

A recursive search finds all `li` tags anywhere.

In [ ]:
document.find_all("li")

### Finding tags by attributes

Sometimes the easiest way to find a tag is by its attribute name. In our examples, both lists have an `id` attribute that uniquely identifies the tables. We can also use the `find*` methods to search for attributes:


In [ ]:
document.find(attrs={"id": "unordered_list"})

### Accessing attributes

The `ul` tag also contains a `style` attribute. Any bs4 tag behaves like a dictionary with attribute names as keys and attribute values as values:

In [ ]:
ulist.attrs

In [ ]:
ulist["style"]

# Downloading a table from Wikipedia

We aim to get a list of countries sorted by their population size:
https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population

First, let's import the required modules:

In [ ]:
import re

import dateutil
import requests
from bs4 import BeautifulSoup

This time, we load the html directly from a website using the requests module:

In [ ]:
r = requests.get("https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population")

The web server returns a status code to indicate if the request was (un-)succesfully.
We use that status-code to check if the page was succesfully loaded:

In [ ]:
assert r.status_code == 200

Next, we extract the html source and initiated BeautifulSoup:

In [ ]:
html = r.text
document = BeautifulSoup(html, "html.parser")

by looking at the document, we can see that we are interested in first table. So we use `find`:

In [ ]:
table = document.find("table")

If you are not familiar with html table, read this example first: https://www.w3schools.com/html/tryit.asp?filename=tryhtml_table_intro

In [ ]:
print(str(table)[:1024])

At this point, it is a good idea to programatically check that the table contains the correct header:

In [ ]:
assert table.find("th").get_text() == "Rank"

In [ ]:
rows = table.find_all("tr")

In [ ]:
for row in rows[1:]:
    cells = row.find_all(["th", "td"])
    if not cells:
        continue

    cells_text = [cell.get_text(strip=True) for cell in cells]
    (rank, country, population, percentage, updated_at, source, comment) = cells_text

    # Clean population string str("1,404,890,600") -> int(1404890600)
    population = int(population.replace(",", ""))

    # Clean country name
    country = re.findall(r"[\w\s()\.]+", country)[0]

    # Convert percentage to floats
    percentage = float(re.findall(r"[\d\.]+", percentage)[0])

    # Convert updated_at to date object
    updated_at = dateutil.parser.parse(updated_at).date()

    print(f"{rank}, {country}, {population:.2e}, {percentage}%, {updated_at}, {source}")

**Attention**: Beautiful Soup does not execute Javascript. This means that you the code in your browser inspector might look a bit different from the original HTML source code. 

# Another example of downloading a Wikipedia table 

Let's consider another table in a Wikipedia page. This page has a lot more tables, so one challenge will be to pick the right table

https://en.wikipedia.org/wiki/Serena_Williams


We are interested in extracting these two tables:

![Target Wikipedia tables](pictures/wiki_tables.png)

**Exercise**: 

Find the tables on a page by locating heading and using `.find_next()`


We begin by downloading the webpage and instatiating the BeautifulSoup object:

In [ ]:
r = requests.get("https://en.wikipedia.org/wiki/Serena_Williams")
document = BeautifulSoup(r.text, "html.parser")

This page contains a lot of tables without specific attributes that would make it easy to find our table of interest. Further, the same headings of the tables are used for multiple tables, making it difficult to find a table just by its headings:

In [ ]:
len(document.find_all("table"))

Therefore, we choose another strategy.

First, we find the tag with class `mw-headling` whose `string` content _starts with_ `Singles`.
Then we find the _next_ table using `heading_element.find_next(...)`:

In [ ]:
# string class
singles_heading = document.find(...
singles_heading

In [ ]:
singles_heading.find_next(...

Now, our tables of interest are the first two result tables for "Singles" and "Women's doubles". We write a small helper function that returns a table with a given heading:

In [ ]:
def find_table_with_heading(document, heading_pat):
    heading_element = document.find(...
    table = heading_element.find_next(...
    return table


In [ ]:
singles_table = find_table_with_heading(document, ...)
# print headers


Next, we can find the table after the heading "Women's doubles"

In [ ]:
doubles_table = find_table_with_heading(document, ...)
# print headers


## Exercise:

- Iterate through the rows
- convert year to integer (or date)
- strip note '(12)' from event, so the same event has the same string
- load into pandas DataFrame (more on pandas in a later lecture)

In [ ]:
data = []
for row in ...:
    cells = ...
    if not cells:
        # e.g. header row
        continue
    values = ...
    # convert year to int
    ...
    # strip trailing (n) notes from events
    ...
    data.append(values)
data

When data is in this form, we can convert it into a DataFrame with pandas.

You'll learn more about pandas next week.

In [ ]:
import pandas as pd

headings = ...
df = pd.DataFrame(data, columns=headings)
df

With pandas, we can filter this data, group it, and plot interesting relationships.

Pandas `groupby` is an interesting operation for performing aggregations,
e.g. counting the wins/losses by year and result:

In [ ]:
results_by_year = df.groupby(...
results_by_year

Which we can now plot

In [ ]:
results_by_year.plot(kind="bar")

Is there any significance to the court?

In [ ]:
results_by_surface = df.groupby(...
results_by_surface

In [ ]:
results_by_surface.plot(kind="bar")

We can even filter to e.g. select opponents who Williams faced at least twice

In [ ]:
results_by_op = df.groupby(...)

# we can exclude opponents only met once:
results_by_op = results_by_op.fillna(0)
multiple_meetings = ...
multiple_meetings.plot(kind="bar")


# Exercise:

Find images on the UiO page

1) Go to https://en.wikipedia.org/wiki/University_of_Oslo 
2) Download the content from the site using BeautifulSoup and requests
3) Search for all images (using `images = document.find_all('img')`) and print out the content
4) Include only images with the attribute `class_="thumbimage"` in your list of images.
5) Print out a list of the value of the "src" attribute for the images in 4. 
6) See if you can display an image by pasting a result from 5 into your web-browser.

In [ ]:
r = requests.get...